# Clustering neighborhoods

In [2]:
!pip install folium
!pip install geopy
import requests
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

     |████████████████████████████████| 93 kB 3.0 MB/s  eta 0:00:01


# reading the wiki using read_html /pandas

# 

In [18]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header=0)[0]
df = df[~df['Neighbourhood'].isin(['Not assigned'])]
df = df[~df['Borough'].isin(['Not assigned'])]

In [19]:
df1[['Postal Code','Borough' , 'Neighborhood']] = df[['Postal Code','Borough','Neighbourhood']]
df1.index = pd.RangeIndex(len(df))

In [20]:
df1.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [22]:
df1.shape

(103, 3)

# Here pls note have used csv instead geopy to get latitude/longtitude.

In [23]:
#read csv file.
df2 = pd.read_csv('http://cocl.us/Geospatial_data')

In [26]:
#Merge the columns by referring lookup -Postal code.
df_merge_col = pd.merge(df1, df2, on='Postal Code')
df_merge_col.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# lets do the clustering process for toronto.

In [28]:
#Lets group the clustering process
toronto_data = df_merge_col[df_merge_col['Borough'].astype(str).str.contains('Toronto')].reset_index(drop=True)
toronto_data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [43]:
#address is Toronto and finding latitude/longtitude to plot a map
address = 'Toronto'
geolocator = Nominatim(user_agent="ny1_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [30]:
#lets find latitude and longtitue for the city Toronto using geolocator.
address = 'Toronto'

geolocator = Nominatim(user_agent="My_Toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))
#lets visualize city lat/long and manhattan_data (Dataframe) using folium
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan
#what we are seeing below is just finding borough,neighborhood ,lat/long of Manhattam city.  We can see 40 neighborhood.

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
CLIENT_ID = 'M5MMNQYOR0ZV0AF1FIIS1QA52ZFUW0FWJR2CSLGHIWWEJHFU' # your Foursquare ID
CLIENT_SECRET = '1ATTQXF425HHMT0XNA2ADBIAUINRXIUWGUEZUYCLBCGBIEGC' # your Foursquare Secret
ACCESS_TOKEN = 'ZLGWM3JXMZVZVS2JJEHTA3CAQUHZDPN22VNCJWLU30QURM0K' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print(toronto_data.loc[0, 'Neighborhood'])
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude'])

Your credentails:
CLIENT_ID: M5MMNQYOR0ZV0AF1FIIS1QA52ZFUW0FWJR2CSLGHIWWEJHFU
CLIENT_SECRET:1ATTQXF425HHMT0XNA2ADBIAUINRXIUWGUEZUYCLBCGBIEGC
Regent Park, Harbourfront
Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerh

In [34]:
toronto_onehots = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
print(toronto_onehots.shape)
# add neighborhood column back to dataframe
toronto_onehots['Neighborhood'] = toronto_venues['Neighborhood'] 
print(toronto_onehots.shape)
# move neighborhood column to the first column
fixed_columns = [toronto_onehots.columns[-1]] + list(toronto_onehots.columns[:-1])
toronto_onehots = toronto_onehots[fixed_columns]


#print(toronto_venues.head())
toronto_onehots.head()

(856, 202)
(856, 202)


,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
toronto_grouped = toronto_onehots.groupby('Neighborhood').mean().reset_index()
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Beer Bar,Seafood Restaurant,Cocktail Bar,Farmers Market,Art Gallery,Basketball Stadium,Liquor Store,Bakery,Bistro,Café
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Bakery,Convenience Store,Restaurant,Nightclub,Italian Restaurant,Intersection,Bar
2,"Business reply mail Processing Centre, South C...",Garden Center,Fast Food Restaurant,Park,Pizza Place,Comic Shop,Burrito Place,Recording Studio,Restaurant,Brewery,Light Rail Station
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Plane,Coffee Shop,Boat or Ferry,Rental Car Location,Boutique,Harbor / Marina,Sculpture Garden
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Seafood Restaurant,Bubble Tea Shop,Spa,Portuguese Restaurant,Poke Place,Comic Shop
5,Christie,Grocery Store,Café,Park,Italian Restaurant,Baby Store,Restaurant,Candy Store,Athletics & Sports,Nightclub,Coffee Shop
6,Church and Wellesley,Sushi Restaurant,Japanese Restaurant,Café,Ramen Restaurant,Escape Room,Creperie,Juice Bar,Burger Joint,Pub,Beer Bar
7,"Commerce Court, Victoria Hotel",Café,Coffee Shop,Restaurant,Gastropub,Gym,American Restaurant,Sandwich Place,Hotel,Beer Bar,Seafood Restaurant
8,Davisville,Dessert Shop,Italian Restaurant,Coffee Shop,Sandwich Place,Café,Pizza Place,Sushi Restaurant,Gym,Indian Restaurant,Brewery
9,Davisville North,Food & Drink Shop,Hotel,Park,Pizza Place,Sandwich Place,Breakfast Spot,Gym / Fitness Center,Department Store,Convenience Store,Diner


In [39]:
#k-Means clustering
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
from sklearn.cluster import KMeans
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels

neighborhoods_sorted = neighborhoods_venues_sorted

neighborhoods_sorted.reset_index(drop=True)
neighborhoods_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

toronto_merged = pd.merge(toronto_merged,neighborhoods_sorted,on='Neighborhood')


In [42]:
#create a map -visualization.
#Finally, let's visualize the resulting clusters

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters